In [ ]:
from utils.io import gdf_viewer, gdf_merger, na_col_drop, na_line_drop, dble_col_drop, col_ren
import re, os
import numpy as np
import pandas as pd

# Pollutants concentrations standard

In [ ]:
tmp_dir='../../CF_data/Result_traitem/Memoris_seafile/'
sheet='donnees'

## Standard values for soil and water 

In [ ]:
df = pd.read_excel('../../CF_data/Data_UMONS/Contamination/Résultats et mesures Siterem/Ensemble des résultats Memoris version Seafile.xls', 
                   sheet_name='Données')#, skiprows=2)
df.replace(r'<|>','', inplace=True, regex=True)
df.replace(r'-$',np.nan, inplace=True, regex=True)
df.drop(columns=df.columns[-3:], inplace=True)

gdf_viewer(df, rows=5, cols=12)

In [ ]:
df.replace("[v|V]aleur(?:s)? de référence","VR", regex=True, inplace=True)
df.replace("[v|V]aleur(?:s)? seuil","VS", regex=True, inplace=True)
df.replace("[v|V]aleur(?:s)? d'intervention","VI", regex=True, inplace=True)

In [ ]:
df.insert(1, "Groupe", np.nan)
f = None
for i in df.index:
    if not pd.isnull(df.iloc[i, 0]) and pd.isnull(df.iloc[i,2]):
        f = df.iloc[i,0]
    if f is not None: 
        df.loc[i, 'Groupe'] = f

In [ ]:
catg= None
v = None
df.loc[63, :] = np.nan # create a new line at the end
for c in range(len(df.columns)):
    if not pd.isnull(df.iloc[0,c]):
        v = df.iloc[0,c]
    if not pd.isnull(df.iloc[1,c]):
        catg= df.iloc[1,c].split(" ")[0]
        df.iloc[0,c] = v + '_' + catg

In [ ]:
df.iloc[0,0] = "Polluant"
df.iloc[0,1] = "Groupe"
df = col_ren(df, line_to_col=0, mode=0)

In [ ]:
df.drop(index=[0] + list(df.query('Polluant == Groupe').index), inplace=True)
df = df.query('Polluant == Polluant')
df.reset_index(drop=True, inplace=True)

In [ ]:
soil_std = df.iloc[:,range(0,14)]
water_std = df.iloc[:,[0,1]+list(range(14,18))]

In [ ]:
gdf_viewer(soil_std, rows=5), gdf_viewer(water_std, rows=5)

In [ ]:
if not os.path.exists(tmp_dir):
    os.makedirs(tmp_dir)
    
soil_std.to_csv(tmp_dir+sheet+'_Pol_Standards_S.csv', index=False)
water_std.to_csv(tmp_dir+sheet+'_Pol_Standards_W.csv', index=False)

## Standard values for air

In [ ]:
df = pd.read_excel('../../CF_data/Data_UMONS/Contamination/Résultats et mesures Siterem/Ensemble des résultats Memoris version Seafile.xls', 
                   sheet_name='Résult air')#, skiprows=2)
df.replace(r'<|>','', inplace=True, regex=True)
df.replace(r'-$',np.nan, inplace=True, regex=True)

gdf_viewer(df, rows=5, cols=12)

In [ ]:
df = df.iloc[:, list(range(3,9))][:132]
df.drop(index=[0,1,2]+list(range(4,34)), inplace=True)
df.reset_index(drop=True, inplace=True)

In [ ]:
df.insert(1, "Groupe", np.nan)
f = None
for i in df.index:
    if not pd.isnull(df.iloc[i, 0]) and pd.isnull(df.iloc[i,2]):
        f = df.iloc[i,0]
    if f is not None: 
        df.loc[i, 'Groupe'] = f

In [ ]:
gdf_viewer(df, rows=5)